In [12]:
import tensorflow as tf
from tensorflow import keras
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt


from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import pandas as pd

import scipy

%matplotlib inline

# Make better use of Jupyter Notebook cell width

from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))
pd.options.display.float_format = '{:.2f}'.format

In [13]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
train_ds = train_gen.flow_from_directory(
    './train/',
    target_size=(150, 150),
    batch_size= 32
)

Found 1594 images belonging to 2 classes.


In [16]:
# Using for loop with next function:

X, y = next(train_ds)

In [17]:
train_ds.class_indices

{'dino': 0, 'dragon': 1}

In [18]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = train_gen.flow_from_directory(
    './test/',
    target_size=(150, 150),
    batch_size= 32
)

Found 394 images belonging to 2 classes.


# For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

- You need to develop the model with following structure:

- The shape for input should be (150, 150, 3)
- Next, create a convolutional layer (Conv2D):
- Use 32 filters
- Kernel size should be (3, 3) (that's the size of the filter)
- Use 'relu' as activation
- Reduce the size of the feature map with max pooling (MaxPooling2D)
- Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using a Flatten layer
- Next, add a Dense layer with 64 neurons and 'relu' activation
- Finally, create the Dense layer with 1 neuron - this will be the output
- The output layer should have an activation - use the appropriate activation for the binary classification case
- As optimizer use SGD with the following parameters:

- SGD(lr=0.002, momentum=0.8)

In [22]:
# Create the model:

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Use SGD optimizer with learning rate of 0.002 and momentum of 0.8

optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Question 1:

- Since we have a binary classification problem, what is the best loss function for us?

- Note: since we specify an activation for the output layer, we don't need to set from_logits=True

In [24]:
# Answer is binary_crossentropy!

# Question 2

- What's the total number of parameters of the model? You can use the summary method for that.

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

In [25]:
# Answer is 11,215,873!

# Generators and Training
- For the next two questions, use the following data generator for both train and test sets:

- ImageDataGenerator(rescale=1./255)
- We don't need to do any additional pre-processing for the images.
- When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
- Use batch_size=20
- Use shuffle=True for both training and test sets.
- For training use .fit() with the following params:

- model.fit(
-    train_generator,
-    epochs=10,
-    validation_data=test_generator
- )

In [26]:
# Creating training parameters

train_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    './train/',
    target_size=(150, 150),
    batch_size= 20,
    shuffle=True,
    class_mode='binary'
)

# Creating validation(test) parameters:

val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    './test/',
    target_size=(150, 150),
    batch_size= 20,
    shuffle=True,
    class_mode='binary'
)

# fitting the model:

model.fit(train_ds, epochs=10, validation_data=val_ds)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.
Epoch 1/10


2023-02-26 13:04:02.179143: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-26 13:04:02.847408: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-26 13:04:02.848431: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-26 13:04:02.848471: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-26 13:04:02.849500: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-26 13:04:02.849617: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


80/80 [==============================] - 11s 109ms/step - loss: 0.6673 - accuracy: 0.6255 - val_loss: 0.6147 - val_accuracy: 0.6193
Epoch 2/10
80/80 [==============================] - 9s 108ms/step - loss: 0.5360 - accuracy: 0.7597 - val_loss: 0.4610 - val_accuracy: 0.8325
Epoch 3/10
80/80 [==============================] - 9s 108ms/step - loss: 0.4319 - accuracy: 0.8156 - val_loss: 0.4833 - val_accuracy: 0.7716
Epoch 4/10
80/80 [==============================] - 9s 108ms/step - loss: 0.3693 - accuracy: 0.8513 - val_loss: 0.3810 - val_accuracy: 0.8223
Epoch 5/10
80/80 [==============================] - 9s 108ms/step - loss: 0.3328 - accuracy: 0.8588 - val_loss: 0.3414 - val_accuracy: 0.8604
Epoch 6/10
80/80 [==============================] - 9s 108ms/step - loss: 0.3007 - accuracy: 0.8858 - val_loss: 0.3321 - val_accuracy: 0.8579
Epoch 7/10
80/80 [==============================] - 9s 109ms/step - loss: 0.2828 - accuracy: 0.8833 - val_loss: 0.3929 - val_accuracy: 0.8173
Epoch 8/10
80/80

# Question 3

- What is the median of training accuracy for all the epochs for this model?

In [39]:
np.median(model.history.history['accuracy'])

0.8710790276527405

# Question 4

- What is the standard deviation of training loss for all the epochs for this model?

In [40]:
np.std(model.history.history['loss'])

0.14017966098234794

# Data Augmentation
- For the next two questions, we'll generate more data using data augmentations.

- Add the following augmentations to your training data generator:

- rotation_range=40,
- width_shift_range=0.2,
- height_shift_range=0.2,
- shear_range=0.2,
- zoom_range=0.2,
- horizontal_flip=True,
- fill_mode='nearest'

In [46]:
# Create image generator for train data and also augment the images:

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    './train/',
    target_size=(150, 150),
    batch_size= 32,
    shuffle=True,
    class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255)

val_ds = val_gen.flow_from_directory(
    './test/',
    target_size=(150, 150),
    batch_size= 32,
    shuffle=True,
    class_mode='binary'
)

# model training with augmentation:

model.fit(train_ds, epochs=10, validation_data=val_ds)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.
Epoch 1/10
50/50 [==============================] - 15s 310ms/step - loss: 0.2876 - accuracy: 0.8745 - val_loss: 0.5714 - val_accuracy: 0.7766
Epoch 2/10
50/50 [==============================] - 15s 306ms/step - loss: 0.2760 - accuracy: 0.8921 - val_loss: 0.6115 - val_accuracy: 0.7741
Epoch 3/10
50/50 [==============================] - 15s 307ms/step - loss: 0.2936 - accuracy: 0.8770 - val_loss: 0.5188 - val_accuracy: 0.8096
Epoch 4/10
50/50 [==============================] - 15s 305ms/step - loss: 0.3073 - accuracy: 0.8777 - val_loss: 0.3088 - val_accuracy: 0.8680
Epoch 5/10
50/50 [==============================] - 15s 306ms/step - loss: 0.2957 - accuracy: 0.8764 - val_loss: 0.3941 - val_accuracy: 0.8452
Epoch 6/10
50/50 [==============================] - 15s 307ms/step - loss: 0.2930 - accuracy: 0.8802 - val_loss: 0.3515 - val_accuracy: 0.8503
Epoch 7/10
50/50 [==============================] - 15s 304

# Question 5

- Let's train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

- What is the mean of test loss for all the epochs for the model trained with augmentations?

In [47]:
np.mean(model.history.history['val_loss'])

0.44137109220027926

# Question 6

- What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [49]:
np.mean(model.history.history['val_accuracy'][5:10])

0.8345177650451661

In [50]:
##